In [1]:
import numpy as np
import pandas as pd

C:\Users\mateu\AppData\Local\Temp\ipykernel_12052\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
from scipy.stats import chi2_contingency, pointbiserialr

class Correlation:
    M_ = None
    
    def __init__(self, column1:list, column2:list, data:pd.DataFrame):
        self.columnNames1 = column1
        self.columnNames2 = column2
        self.data         = data
        self.tupla        = ( len(self.columnNames1) , len(self.columnNames2) ) 

    def quiQuadrada(self):
        self.M_ = np.zeros(shape=self.tupla)
        
        for enumI, i in enumerate(self.columnNames1):
            for enumJ, j in enumerate(self.columnNames2):
                chi, p, _, _ = chi2_contingency( pd.crosstab(self.data[i].values, self.data[j].values) )
                self.M_[enumI,enumJ] = p
        
        return pd.DataFrame(self.M_, columns=self.columnNames1, index=self.columnNames2)

    def pointBisserl(self):
        self.M_ = np.zeros(shape=self.tupla)
        for enumI, i in enumerate(self.columnNames1):
            for enumJ, j in enumerate(self.columnNames2):
                corr, p_value = pointbiserialr(self.data[i].values, self.data[j].values)
              
                self.M_[enumI, enumJ] = p_value
        return pd.DataFrame(self.M_.T, columns=self.columnNames1, index=self.columnNames2)

Ao uso de redes neurais e verificar a proporção de classes , e utilizar o hiperparametros `class_weight` para atribuir peso as classes

In [ ]:
from collections import Counter

def calcular_pesos_classes(y):
    # Contar o número de amostras de cada classe
    contagem_classes = Counter(y)
    
    # Calcular o número total de amostras
    total_amostras = sum(contagem_classes.values())
    
    # Calcular a proporção de cada classe
    proporcoes_classes = {classe: total_amostras / contagem for classe, contagem in contagem_classes.items()}
    
    # Calcular os pesos das classes
    pesos_classes = {classe: total_amostras / (len(contagem_classes) * contagem) for classe, contagem in contagem_classes.items()}
    
    return pesos_classes

In [ ]:
from scipy.stats             import pointbiserialr
from scipy.stats.contingency import chi2_contingency, association
from sklearn.preprocessing   import LabelEncoder

class Correlation:
    M_ = None
    def __init__(self, column1:list, column2:list, data:pd.DataFrame):
        self.data = copy.deepcopy(data)
        self.column_names1 = column1
        self.column_names2 = column2
        self.M_ = np.zeros(shape=(len(column1) , len(column2)))

    # Cat-Cat
    def cramerV(self):
        self.data.dropna(inplace=True)

        for enum_i, i in enumerate(self.column_names1):
            for enum_j, j in enumerate(self.column_names2):

                values_i = self.data[i].values
                values_j = self.data[j].values

                if values_i.dtype=='object' or values_i.dtype==pd.Categorical:
                    values_i =  self.Encoder(values_i)
                if values_j.dtype=='object' or values_j.dtype==pd.Categorical:
                    values_j =  self.Encoder(values_j)

                subset = pd.crosstab( values_i, values_j)
                metrica = association(observed=subset, method='cramer')
                self.M_[enum_i, enum_j] = metrica

        return pd.DataFrame( data=self.M_.T, columns=self.column_names1,
                             index=self.column_names2 )
    # Categorico-Numericos
    def pointBisserl(self):
        self.data.dropna(inplace=True)


        for enum_i, i in enumerate(self.column_names1):
            for enum_j, j in enumerate(self.column_names2):

                values_i = self.data[i].values
                values_j = self.data[j].values

                if values_i.dtype=='object' or values_i.dtype==pd.Categorical:
                    values_i =  self.Encoder(values_i)
                if values_j.dtype=='object' or values_j.dtype==pd.Categorical:
                    values_j =  self.Encoder(values_j)

                corr, _ = pointbiserialr(values_i, values_j)
                self.M_[enum_i, enum_j] = corr

        return pd.DataFrame( data=self.M_.T, columns = self.column_names1,
                             index=self.column_names2 )

    def Encoder(self, X): return LabelEncoder().fit_transform(X)